---
<div align="center">

# LSDS | Machine Learning Pipeline
</div>

---
## Project Overview
---

### Background Overview
In critical care, accurately predicting the length of a patient’s stay in the ICU is essential for optimizing resource allocation and improving patient outcomes. The challenge stems from the complex interplay of diverse clinical factors — ranging from vital signs and laboratory results to administered treatments — which requires a concerted effort between clinical expertise and advanced data analytics to develop a robust solution.

---

### Available Data
The [MIMIC-III](https://physionet.org/content/mimiciii/1.4/) dataset, curated by the Massachusetts Institute of Technology, is one of the most comprehensive open-access databases for critical care research. It offers a wealth of de-identified data, including patient demographics, vital signs, lab results, and treatment details, which are invaluable for developing machine learning models.

---

### Approach / Methodology
We will begin by isolating the most relevant data from the MIMIC dataset using Google Cloud’s BigQuery service, which is well-suited to handle such high-volume datasets efficiently. Through extensive Exploratory Data Analysis (EDA), we aim to identify the critical features and patterns that influence patient outcomes. This initial phase will guide the selection of pertinent variables, setting the foundation for a robust predictive model.

---

### Machine Learning Pipeline
The final stage involves constructing a comprehensive machine learning pipeline. This pipeline will cover all steps from data preprocessing and feature engineering to model training and validation. By integrating profiling throughout our workflow, we ensure that performance metrics—such as execution time and resource usage—are continually monitored, a crucial aspect given the volume of data we are working with. This approach will help us fine-tune our chosen machine learning algorithm, ultimately providing a dependable prediction model for ICU length of stay.

---

### Conclusions
In conclusion, by leveraging BigQuery for efficient data extraction and performing a comprehensive exploratory analysis of the MIMIC dataset, we hope to develop a robust machine learning pipeline to predict ICU length of stay. This project aims to achieve more accurate and timely predictions, ultimately supporting improved resource allocation, enhanced clinical decision-making, and better patient outcomes in critical care settings.

---

### Additional Considerations
Each step of the pipeline — from data extraction and preprocessing to model training and validation—has been carefully profiled to monitor execution time, resource usage, and potential bottlenecks. This systematic profiling not only ensures that our solution scales effectively with high volumes of data but also provides valuable insights for continuous optimization of the overall process.

---
## Project Dependencies
---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Remove Warnings
import warnings
warnings.filterwarnings('ignore')
from time import (time)

In [3]:
# Get the starting time before importing the modules
startTime = time()

# Import modules
import numpy as np
import pandas as pd
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.sql.types import ArrayType, StringType, BooleanType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from google.cloud import storage
from google.cloud import bigquery
from pprint import pprint

# Custom Imports
from Utils import (loadConfig, loadPathsConfig)
from DataPreprocessing.DatasetManager import (DatasetManager)
from DataPreprocessing.DaskDataFrame import (DaskDataframe)
from DataPreprocessing.DatasetAnalyser import (DatasetAnalyser)

# Print the amount of time it took to import everything
print(f"Took{(time() - startTime) : .3e}(s) to import all the modules!")

Took 4.695e+00(s) to import all the modules!


In [4]:
# Load the configs
config = loadConfig()
pathsConfig = loadPathsConfig()

In [5]:
# Print the configs
pprint(config)
pprint(pathsConfig)

{'seed': 14, 'targetFeature': 'LOS'}
{'Datasets': {'ADMISSIONS': './Datasets/ADMISSIONS.csv',
              'CHARTEVENTS': './Datasets/CHARTEVENTS.csv',
              'DIAGNOSES_ICD': './Datasets/DIAGNOSES_ICD.csv',
              'ICUSTAYS': './Datasets/ICUSTAYS.csv',
              'PATIENTS': './Datasets/PATIENTS.csv'}}


In [6]:
# Define the path to the environment key
key = './ml-pipeline-key.json'

---
<div align="center">

## Exploratory Data Analysis
</div>

---

Given the MIMIC dataset, we aim to obtain greater insights on how the data is organized within the multiple tables and gain a better understanding on how we can harness the data to properly train a Machine Learning Model to help us correctly predict the Length of Stay (LOS) of a given patient that entered the Intensive Care Unit (ICU).

---
### MIMIC-III Dataset
---

The [MIMIC-III](https://physionet.org/content/mimiciii/1.4/) Dataset contains multiple tables from which we are going to select the ones that provide a greater value to our project and further analyse them to better gain a better understanding of the available data. Furthermore, its important to notice that we have uploaded the tables from the dataset onto the google cloud BigQuery Service to help us properly manage these high volumes of data.

Since our project focuses on predicting ICU length of stay, MIMIC provides a robust platform to explore complex clinical patterns and build predictive models that can help optimize patient care and resource allocation.

> MENTION ALL THE TABLES ON THE DATASET AND WHY WE CHOSE THESE OVER THE OTHERS.

Consequently, we ended up selecting the following tables:

- ADMISSIONS
- DIAGNOSES_ICD
- ICUSTAYS
- PATIENTS

In [7]:
# Create a Client to query the data
client = bigquery.Client.from_service_account_json(key)

---
#### ADMISSIONS Table
---
> The ADMISSIONS table gives information regarding a patient’s admission to the hospital. Since each unique hospital visit for a patient is assigned a unique HADM_ID, the ADMISSIONS table can be considered as a definition table for HADM_ID. Information available includes timing information for admission and discharge, demographic information, the source of the admission, and so on.

For the Admissions Table we selected the following columns:

| ADMISSIONS           | Description                                                                                       |
|:--------------------:|:-------------------------------------------------------------------------------------------------:|
| ADMISSION_TYPE       | Describes the nature of the admission (e.g., ELECTIVE, URGENT, NEWBORN, EMERGENCY)                |
| ADMITTIME            | Timestamp indicating when the patient was admitted to the hospital                                |
| DISCHTIME            | Timestamp indicating when the patient was discharged from the hospital                            |
| DEATHTIME            | Timestamp recording the time of in-hospital death, if applicable                                  |
| INSURANCE            | Contains the insurance provider details for the patient                                           |
| LANGUAGE             | Indicates the language spoken by the patient                                                      |
| RELIGION             | Provides the patient's religious affiliation                                                      |
| MARITAL_STATUS       | Describes the patient's marital status                                                            |
| ETHNICITY            | Specifies the patient's ethnic background                                                         |
| HOSPITAL_EXPIRE_FLAG | Binary indicator (1 or 0) showing whether the patient died during the hospital stay               |

---
#### DIAGNOSES_ICD Table
---

> The DIAGNOSES_ICS table contains ICD diagnoses for patients, most notably ICD-9 diagnoses.

We have selected the following columns from this table:

| DIAGNOSES_ICD | Description                                                                                                                          |
|:-------------:|:------------------------------------------------------------------------------------------------------------------------------------:|
| SUBJECT_ID    | Unique identifier for each patient                                                                                                   |
| HADM_ID       | Unique identifier for each hospital admission, defining a specific patient stay                                                      |
| SEQ_NUM       | Indicates the order or priority of the ICD diagnoses for the patient, which can affect treatment reimbursement                       |
| ICD9_CODE     | Contains the ICD-9 code corresponding to the diagnosis assigned to the patient during the hospital stay                              |

---
#### ICUSTAYS Table
---

> The ICUSTAYS defines each ICUSTAY_ID in the database, i.e. defines a single ICU stay.

We have selected the following columns from this table:

| ICUSTAYS          | Description                                                                                                                             |
|:-----------------:|:---------------------------------------------------------------------------------------------------------------------------------------:|
| SUBJECT_ID        | Unique identifier for each patient.                                                                                                     |
| HADM_ID           | Unique identifier for each hospital admission, defining a specific patient stay.                                                        |
| ICUSTAY_ID        | Unique identifier for each ICU stay, representing a single ICU encounter.                                                               |
| DBSOURCE          | Indicates the original ICU database source (e.g., 'carevue' for 2001–2008 or 'metavision' for 2008–2012), which affects data archiving. |
| FIRST_CAREUNIT    | The first ICU type where the patient was cared for during the ICU stay.                                                                 |
| LAST_CAREUNIT     | The last ICU type where the patient was cared for, noting that transfers between ICU types can occur within a single stay.              |
| FIRST_WARDID      | Identifier for the first ICU unit (ward) where the patient was admitted, as recorded in the hospital database.                          |
| LAST_WARDID       | Identifier for the last ICU unit (ward) where the patient stayed during their ICU encounter.                                            |
| INTIME            | Timestamp indicating when the patient was transferred into the ICU.                                                                     |
| OUTTIME           | Timestamp indicating when the patient was transferred out of the ICU.                                                                   |
| LOS               | Length of stay for the ICU encounter, measured in fractional days, which may encompass multiple ICU units.                              |

---
#### PATIENTS Table
---

> The PATIENTS table defines each SUBJECT_ID in the database, i.e. defines a single patient.

For this table we have selected the upcoming columns:

| PATIENTS     | Description                                                                                                                                          |
|:------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------:|
| SUBJECT_ID   | Unique identifier for each patient, used to link patient data across tables.                                                                         |
| GENDER       | The genotypical sex of the patient.                                                                                                                  |
| DOB          | Date of birth of the patient. For patients older than 89, DOB is shifted (set to 300 years before the first admission) for HIPAA compliance.         |
| DOD          | Date of death for the patient, merging information from hospital (DOD_HOSP) and social security (DOD_SSN) records, with DOD_HOSP taking priority.    |
| DOD_HOSP     | Date of death as recorded in the hospital database.                                                                                                  |
| DOD_SSN      | Date of death from the social security database.                                                                                                     |
| EXPIRE_FLAG  | Binary flag indicating whether the patient died (i.e., whether DOD is not null).                                                                     |

In [12]:
class QueryPerformer:
    def __init__(self, client):
        self.client = client

    def query(self, queryStatement):
        queryJob = self.client.query(queryStatement)
        df = queryJob.to_dataframe()

In [11]:
# Define your query
query = """
    SELECT ()
    FROM `MIMIC.PATIENTS`
    LIMIT 10
"""

# Run the query
query_job = client.query(query)

# Wait for the job to complete and get the results
results = query_job.result()

# Convert the results to a Pandas DataFrame
admissions = query_job.to_dataframe()

In [9]:
admissions

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00+00:00,NaT,NaT,NaT,0
1,238,253,F,2089-11-26 00:00:00+00:00,NaT,NaT,NaT,0
2,242,258,F,2124-09-19 00:00:00+00:00,NaT,NaT,NaT,0
3,243,260,F,2105-03-23 00:00:00+00:00,NaT,NaT,NaT,0
4,247,264,F,2162-11-30 00:00:00+00:00,NaT,NaT,NaT,0
5,249,266,F,2090-12-17 00:00:00+00:00,NaT,NaT,NaT,0
6,250,267,F,2131-09-05 00:00:00+00:00,NaT,NaT,NaT,0
7,626,663,F,2086-05-12 00:00:00+00:00,NaT,NaT,NaT,0
8,630,667,F,2053-09-17 00:00:00+00:00,NaT,NaT,NaT,0
9,638,675,F,2158-04-05 00:00:00+00:00,NaT,NaT,NaT,0
